In [112]:
import pymorphy2
import requests
from bs4 import BeautifulSoup
import re
from stop_words import get_stop_words

import numpy as np
from operator import itemgetter

### Скачивание страниц и нормализация предложений

In [51]:
class NormalAnalyzer:
    __stemmer = pymorphy2.MorphAnalyzer()
    __cache = {}
    
    def norm(self, token):
        token = token.strip()
        
        norm = ''
        if token not in self.__cache:
    
            res = self.__stemmer.parse(token)
            norm = res[0].normal_form
            self.__cache[token] = norm
        else:
            norm = self.__cache[token]
        return norm

In [193]:
class TextUrl():
    def __init__(self):
        self.sentensesDetect = re.compile(r"[^.!?\s][^.!?]*(?:[.!?](?!['\"]?\s|$)[^.!?]*)*[.!?]?['\"]?(?=\s|$)", re.MULTILINE |  re.DOTALL )
        self.wordsDetect = re.compile(u"[А-Яа-я]+")
        
        self.normal_analizer = NormalAnalyzer()
        self.stop_words = get_stop_words('ru')
        
    def sentenseToWords(self, sent):
        words = self.wordsDetect.findall(sent)
        normal_words = []

        # lemmatize words
        for word in words:
            word = self.normal_analizer.norm(word.lower())
            if word not in self.stop_words:
                normal_words.append(word)
        
        return normal_words
        
    def extractText(self, url):
        html = requests.get(url).text
        soup = BeautifulSoup(html, 'html.parser')
        
        sentenses = []
        realSentenses = []
        for paragraph in soup.find_all('p'):
            text = paragraph.get_text()
            for sent in self.sentensesDetect.findall(text):
                normal_words = self.sentenseToWords(sent)
                
                if len(normal_words) != 0:
                    sentenses.append(normal_words)
                    realSentenses.append(sent)
        
        return sentenses, realSentenses

In [194]:
textUrl = TextUrl()
all_sentenses = []
all_real_sentenses = []
all_urls = ['https://ru.wikipedia.org/wiki/Рабье,_Бенжамен',
            'https://ru.wikipedia.org/wiki/Весёлая_бурёнка',
            'https://ru.wikipedia.org/wiki/Норвежский_чёрный_элкхунд',
            'https://ru.wikipedia.org/wiki/Космические_Юра_и_Нюра',
            'https://ru.wikipedia.org/wiki/Невесомость',
            'https://ru.wikipedia.org/wiki/Союз_МС-04'
           ]
for url in all_urls:
    sent, realSents = textUrl.extractText(url)
    all_sentenses.extend(sent)
    all_real_sentenses.extend(realSents)

### Функции над векторами, представленными словарями

In [106]:
def lenVector(vec):
    return np.linalg.norm(vec.values())

In [107]:
def normolizeVec(vec):
    lenVec = lenVector(vec)
    if lenVec == 0:
        return 0
    return dict(zip(vec.keys(), vec.values() / lenVec))

In [116]:
# Скалярное произведение двух векторов, представленных списками
def dotVec(d1, d2):
    intersection = set.intersection(set(d1.keys()), set(d2.keys()))
    value = 0
    for key in intersection:
        value += d1[key] * d2[key]
    return value

### Составление векторов предложений

In [117]:
# Список слов
all_words = set()
for sent in all_sentenses:
    for word in sent:
        all_words.add(word)

In [206]:
# Вектора представлены словарями (нет слова = 0)
vectors_sent = []
for sent in all_sentenses:
    words = {}
    for word in sent:
        words[word] = words.setdefault(word, 0) + 1
    
    vectors_sent.append(words)   

In [208]:
# Нормализация векторов
vectors_sent_norm = vectors_sent
for i, vec in enumerate(vectors_sent):
    vectors_sent_norm[i] = normolizeVec(vec)

In [120]:
# idf
df = dict(zip(all_words, np.zeros(len(all_words))))
for vec in vectors_sent:
    words = vec.keys()
    for word in words:
        df[word] += 1
        
idf = dict(zip(df.keys(), np.log(float(len(all_words)) / np.array(df.values()))))

### Вектор запроса

In [153]:
queries = [u'Французский художник добился успеха со смеющейся коровой (на илл.) и провалился с пьющими ослами.',
           u'С собакой викингов ходят на лося.',
           u'Герои мультфильма стали индикаторами невесомости космического корабля «Арго».'
          ]

vectors_query = []

for query in queries:
    words = {}
    norm_words = textUrl.sentenseToWords(query)
    for word in norm_words:
        words[word] = words.setdefault(word, 0) + 1
    
    vectors_query.append(words)   

### Векторное ранжирование

In [154]:
def vecRanging(query, sentenses):
    rang = []
    for sent in sentenses:
        rang.append(dotVec(query, sent) / (lenVector(query) * lenVector(sent)))
    return sorted(zip(range(len(rang)), rang), key=itemgetter(1), reverse=True)

In [214]:
def getTopVec(num_query, ranging='vecRanging', N=7):
    if ranging == 'vecRanging':
        serp = vecRanging(vectors_query[num_query], vectors_sent)[:N]
    else:
        serp = tf_idfRanging(vectors_query[num_query], vectors_sent_norm)[:N]
    print u"Запрос: " + queries[num_query]
    print u"Топ выдачи:"
    for result in serp:
        print "\t" + all_real_sentenses[result[0]] + "\t" + str(result[1])
        print "\n"

In [218]:
for i in [0,1,2]:
    getTopVec(i, ranging='vecRanging')
    print '____________________________________'
    print

Запрос: Французский художник добился успеха со смеющейся коровой (на илл.) и провалился с пьющими ослами.
Топ выдачи:
	В середине 1920-х годов на французском рынке сырной продукции появилось множество имитаций «Смеющейся коровы» и разнообразных вариаций на тему этикетки Рабье — «Улыбающаяся корова» (фр. «La vache qui sourit»), «Говорящая корова» (фр. «La vache qui parle»), «Читающая корова» (фр. «La vache qui lit»), «Учёная корова» (фр. «La vache savante»), «Брыкающаяся корова» (фр. «La vache qui rue») и др., в ассортименте были представлены также «Смеющаяся обезьяна» (фр. «Le Singe qui rit»), «Смеющаяся коза» (фр. «La chèvre qui rit»), «Смеющийся кот» (фр. «Le chat qui rit») и др.[15][14]	0.288874152291


	<…> C собакой ещё проходит, но заставить смеяться корову!	0.258198889747


	Весёлая бурёнка (фр. La vache qui rit, с фр. — «Смеющаяся корова») — французский плавленый сыр, производимый группой компаний Bel Group.	0.237170824513


	Рисунок смеющейся коровы был сделан по мотивам виден

### TF-IDF ранжирование

In [202]:
def tf_idfRanging(query, sentenses):
    rang = []
    for sent in sentenses:
        tf_idf = 0
        for word in query.keys():
            if word in sent.keys() and word in idf.keys():
                tf_idf += np.log(sent[word] + 1) * idf[word]
        rang.append(tf_idf)
            
    return sorted(zip(range(len(rang)), rang), key=itemgetter(1), reverse=True)

In [220]:
for i in [0,1,2]:
    getTopVec(i, ranging='tf_idf')
    print '____________________________________'
    print

Запрос: Французский художник добился успеха со смеющейся коровой (на илл.) и провалился с пьющими ослами.
Топ выдачи:
	В середине 1920-х годов на французском рынке сырной продукции появилось множество имитаций «Смеющейся коровы» и разнообразных вариаций на тему этикетки Рабье — «Улыбающаяся корова» (фр. «La vache qui sourit»), «Говорящая корова» (фр. «La vache qui parle»), «Читающая корова» (фр. «La vache qui lit»), «Учёная корова» (фр. «La vache savante»), «Брыкающаяся корова» (фр. «La vache qui rue») и др., в ассортименте были представлены также «Смеющаяся обезьяна» (фр. «Le Singe qui rit»), «Смеющаяся коза» (фр. «La chèvre qui rit»), «Смеющийся кот» (фр. «Le chat qui rit») и др.[15][14]	3.91400857028


	<…> C собакой ещё проходит, но заставить смеяться корову!	3.48565662094


	Весёлая бурёнка (фр. La vache qui rit, с фр. — «Смеющаяся корова») — французский плавленый сыр, производимый группой компаний Bel Group.	3.47039507297


	Рисунок смеющейся коровы был сделан по мотивам виденной